# Introduction
1. We have two `class` to extract the information neighbors from center atoms, they are all derived class of `StructureNeighborsBase`
    1. `matersdk.io.publicLayer.neigh.StructureNeighborsV1`: Use `NearestNeighbors` in `sklearn`
    2. `matersdk.io.publicLayer.neigh.StructureNeighborsV2`: Implemet custom `NearestNeighbors`
    3. Now `StructureNeighborsV2` is much faster than `StructureNeighborsV1`

In [7]:
from timeit import default_timer as timer
from matersdk.io.publicLayer.structure import DStructure
from matersdk.io.publicLayer.neigh import StructureNeighborUtils
from matersdk.io.publicLayer.neigh import StructureNeighborsDescriptor

# 0. Some utils to get information of structure neighbors-- `matersdk.io.publicLayer.neigh.StructureNeighborUtils`

## 0.1. Get `max_nbrs_num_real`

In [11]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]
rcut = 3.2
coords_are_cartesian = True

structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
max_num_nbrs_real = StructureNeighborUtils.get_max_num_nbrs_real(
                        structure=structure,
                        scaling_matrix=scaling_matrix,
                        rcut=rcut,
                        coords_are_cartesian=coords_are_cartesian)
print("1. `max_num_nbrs_real` within rcut({0}) of this structure is : {1}".format(rcut, max_num_nbrs_real))

1. `max_num_nbrs_real` within rcut(3.2) of this structure is : 12


# 1. Demo for `StructureNeighborsV1` (Will be deprecated)
1. Running time: 1.66 s

In [3]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]  # Expand the primitive cell to process pbc
reformat_mark = True    # Resort the atoms according to atomic number after expanding to supercell
n_neighbors = 200   # The total number of all kinds of atoms. You'd better set it larger than `max_num_nbr`(usually 100) in `DeepPot-SE`
algorithm = "ball_tree" # The algorithm for `sklearn.NearestNerghbors`
coords_are_cartesian = True # Use cartesian coordinates to find neighbors

_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
struct_neighbors = StructureNeighborsDescriptor.create(
                    'v1',
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    n_neighbors=n_neighbors,
                    algorithm=algorithm)
_end = timer()

print("\t1.1. The number of atoms in primitive cell:\t", len(struct_neighbors.structure.species))
print("\t1.2. The shape of key_nbr_atomic_numbers:\t", struct_neighbors.key_nbr_atomic_numbers.shape)
print("\t1.3. The shape of key_nbr_distances:\t", struct_neighbors.key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", struct_neighbors.key_nbr_coords.shape)
print("Running time: {0} s".format(_end-_start))

	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_atomic_numbers:	 (12, 200)
	1.3. The shape of key_nbr_distances:	 (12, 200)
	1.4. The shape of key_nbr_coords:	 (12, 200, 3)
Running time: 1.6804661779897287 s


# 2. Demo for `StructureNeighborsV2` (Will be deprecated)
1. Running time: 0.037 s

In [4]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [5, 5, 1]  # Expand the primitive cell to process pbd
reformat_mark = True    # Resort the atoms according to atomic number after expanding to supercell
n_neighbors = 200   # The total number of all kinds of atoms. You'd better set it larger than `max_num_nbr`(usually 100) in `DeepPot-SE`
coords_are_cartesian = True # Use cartesian coordinates to find neighbors

_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
struct_neighbors = StructureNeighborsDescriptor.create(
                    'v2',
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    n_neighbors=n_neighbors)
_end = timer()

print("\t1.1. The number of atoms in primitive cell:\t", len(struct_neighbors.structure.species))
print("\t1.2. The shape of key_nbr_atomic_numbers:\t", struct_neighbors.key_nbr_atomic_numbers.shape)
print("\t1.3. The shape of key_nbr_distances:\t", struct_neighbors.key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", struct_neighbors.key_nbr_coords.shape)
print("Running time: {0} s".format(_end-_start))

	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_atomic_numbers:	 (12, 200)
	1.3. The shape of key_nbr_distances:	 (12, 200)
	1.4. The shape of key_nbr_coords:	 (12, 200, 3)
Running time: 0.025744684971868992 s


# 3. Demo for `StructureNeighborsV3`

## 3.1. If you don't specify the `max_nbrs_num` (`max_nbrs_num=False`).
1. The `max_nbrs_num = max_nbrs_num_real+1`
2. Note: Including center atom self

In [20]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [3, 3, 1]
reformat_mark = True
coords_are_cartesian = True
rcut = 3.2


_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
neighbors_v3 = StructureNeighborsDescriptor.create(
                    'v3',
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    rcut=rcut)
_end = timer()


print()
print("Step 1. primitive_cell 中原子的近邻原子情况:")
print("\t1.1. The number of atoms in primitive cell:\t", len(neighbors_v3.structure.species))
print("\t1.2. The shape of key_nbr_species:\t", neighbors_v3.key_nbr_atomic_numbers.shape)
print("\t1.3. The shape of key_nbr_distances:\t", neighbors_v3.key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", neighbors_v3.key_nbr_coords.shape)

print("Running time: {0} s".format(_end-_start))


Step 1. primitive_cell 中原子的近邻原子情况:
	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_species:	 (12, 13)
	1.3. The shape of key_nbr_distances:	 (12, 13)
	1.4. The shape of key_nbr_coords:	 (12, 13, 3)
Running time: 0.008778406074270606 s


In [23]:
### Print atomic number of neighbor atoms for all central atom in `DStructure`
neighbors_v3.key_nbr_atomic_numbers

array([[16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [16., 42., 42., 42., 16., 16., 16., 16., 16., 16., 16.,  0.,  0.],
       [42., 16., 16., 16., 16., 16., 16., 42., 42., 42., 42., 42., 42.],
       [42., 16., 16., 16., 16., 16., 16., 42., 42., 42., 42., 42., 42.],
       [42., 16., 16., 16., 16., 16., 16., 42., 42., 42., 42., 42., 42.],
       [42., 16., 16., 16., 16., 16., 16., 42., 42., 42., 42., 42., 42.]])

In [25]:
### Print distances of neighbor atoms for all central atom in `DStructure`
#neighbors_v3.key_nbr_distances

## 3.2. If you specify the `max_nbrs_num` (`max_nbrs_num=int`).
1. The `max_nbrs_num = max_nbrs_num`
2. Note: Including center atom self.

In [18]:
atom_config_path = "/data/home/liuhanyu/hyliu/code/matersdk/demo/structure/atom.config"
scaling_matrix = [3, 3, 1]
reformat_mark = True
rcut = 3.2
max_nbrs_num = 44
coords_are_cartesian = True   


_start = timer()
structure = DStructure.from_file(file_format="pwmat", file_path=atom_config_path)
neighbors_v3 = StructureNeighborsDescriptor.create(
                    'v3',
                    structure=structure,
                    scaling_matrix=scaling_matrix,
                    reformat_mark=reformat_mark,
                    coords_are_cartesian=coords_are_cartesian,
                    rcut=rcut,
                    max_nbrs_num=max_nbrs_num
                    )
_end = timer()


print()
print("Step 1. primitive_cell 中原子的近邻原子情况:")
key_nbr_species, key_nbr_distances, key_nbr_coords = \
            neighbors_v3._get_key_neighs_info(
                            scaling_matrix=scaling_matrix,
                            rcut=rcut,
                            coords_are_cartesian=coords_are_cartesian)
print("\t1.1. The number of atoms in primitive cell:\t", len(neighbors_v3.structure.species))
print("\t1.2. The shape of key_nbr_species:\t", key_nbr_species.shape)
print("\t1.3. The shape of key_nbr_distances:\t", key_nbr_distances.shape)
print("\t1.4. The shape of key_nbr_coords:\t", key_nbr_coords.shape)

print("Running time: {0} s".format(_end-_start))


Step 1. primitive_cell 中原子的近邻原子情况:
	1.1. The number of atoms in primitive cell:	 12
	1.2. The shape of key_nbr_species:	 (12, 44)
	1.3. The shape of key_nbr_distances:	 (12, 44)
	1.4. The shape of key_nbr_coords:	 (12, 44, 3)
Running time: 0.006965867010876536 s
